In [ ]:
!git clone https://github.com/isi-nlp/uroman.git
!pip install franco_arabic_transliterator

Cloning into 'uroman'...
remote: Enumerating objects: 168, done.
remote: Total 168 (delta 0), reused 0 (delta 0), pack-reused 168
Receiving objects: 100% (168/168), 4.30 MiB | 16.88 MiB/s, done.
Resolving deltas: 100% (68/68), done.
     |████████████████████████████████| 1.9MB 8.7MB/s 
     |████████████████████████████████| 26.9MB 1.6MB/s 
  Created wheel for franco-arabic-transliterator: filename=franco_arabic_transliterator-0.0.1.5-cp37-none-any.whl size=1997539 sha256=ee54e2076c550562558219e00c601fb5e106fef9f2754e60eb8b273ee8e65f16
  Stored in directory: /root/.cache/pip/wheels/7a/89/fa/bf1d07c67b5f0d46b20e6faf7caab30c019e5108b9de473a44
Successfully built franco-arabic-transliterator


In [ ]:
from franco_arabic_transliterator.franco_arabic_transliterator import FrancoArabicTransliterator
import re
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()

import logging
logging.basicConfig()
logging.getLogger().setLevel(logging.ERROR)

transliterator = FrancoArabicTransliterator()

def preprocess_token(s):
    # Shorten repeated characters to avoid slow transliteration speed
    s = re.sub(r"(\w)(\1){2,}", r"\1\1", s)
    s = re.sub(r"(?:(\w)(\w))(\1\2){2,}", r"\1\2\1\2\1\2", s)
    s = re.sub(r"(?:(\w)(\w)(\w))(\1\2\3){2,}", r"\1\2\3\1\2\3", s)
    # Normalize laughing token
    s = re.sub(r"h[ha]{3,}", "hahaha", s)
    # Truncate the token to max length of 10 characters
    return s[:10]

def convert_arabiz2arabic(text):
    tokens = text.split()
    tokens = [preprocess_token(t) for t in tokens]
    transliterated_tokens = [transliterator.transliterate(t, method="lexicon") for t in tokens]
    return " ".join(transliterated_tokens)

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [ ]:
train_df = pd.read_csv("../input/Train.csv")
test_df = pd.read_csv("../input/Test.csv")

In [ ]:
%%time
train_df['text'] = train_df['text'].progress_apply(lambda x: convert_arabiz2arabic(x))
test_df['text'] = test_df['text'].progress_apply(lambda x: convert_arabiz2arabic(x))

train_df.to_csv('../input/train_trans.csv')
test_df.to_csv('../input/test_trans.csv')


CPU times: user 9min 51s, sys: 14.7 s, total: 10min 6s
Wall time: 10min 10s
